In this notebook, I solve the alternative HW1 for Kyle Herkenhoff's . In this homework, we are asked to solve a simple Bewley model.

In [2]:
using LinearAlgebra, Optim, Makie, Plots

In [1]:
 # Params = [beta, gamma, number_of_states, number_of_grid_points]
Params = [0.9, 1.0, 2, 101]

2-element Vector{Float64}:
 0.9
 1.0

In [ ]:
function states(P)

    n = P[3]



end

In [ ]:
function expanding_grid(P)

    n = P[4]
    

end

In [3]:
function utility(c, P)

    if P[2] == 1

        return log(c)

    else

        return c^(1-P[2])/(1-P[2])

    end

end

utility (generic function with 1 method)